In [8]:
from flask import Flask, render_template, request
import sqlite3
import pandas as pd
from betting_functions import load_api_key, fetch_sports_markets, save_to_sql

app = Flask(__name__)

# Connect to the SQLite database (replace with your own database file)
db_path = 'betting_data.db'

# Load the API Key from file
api_key = load_api_key('api_key.txt')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/results', methods=['POST'])
def results():
    # Get input data from the form
    sport = request.form['sport']
    league = request.form['league']
    market_type = request.form['market_type']

    # Fetch the game and player markets using the input parameters
    game_ids = get_todays_game_ids(api_key, league)
    game_data = fetch_game_data(game_ids, api_key, market_type=market_type, sport=sport, league=league, sportsbooks=['Pinnacle', 'Caesars'])

    # Filter data and calculate EV
    ev_bets = find_plus_ev_bets(game_data, threshold=5)

    # Save unfiltered data to the SQL database
    conn = sqlite3.connect(db_path)
    save_to_sql(game_data, league, market_type, conn)
    conn.close()

    # Return the filtered data to the frontend
    return render_template('results.html', tables=[ev_bets.to_html(classes='data', header="true")])

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/colesprouse/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/colesprouse/anaconda3/lib/python3.11/site-packages/ipykernel/__init__.py", line 5, in <module>
    from .connect import *  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/colesprouse/anaconda3/lib/python3.11/site-packages/ipykernel/connect.py", line 11, in <module>
    import jupyter_client
  File "/Users/colesprouse/anaconda3/lib/python3.11/site-packages/jupyter_client/__init__.py", line 3, in <module>
    from .asynchronous import AsyncKernelClient
  File "/Users/colesprouse/anaconda3/lib/python3.11/site-packages/jupyter_client/asynchronous/__init__.py", line 1, in <module>
    from .client import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/colesprou

SystemExit: 1